## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
<p>
    <font color="#120a8f">
Данные, с которыми предстоит работать:
        <ul>
            <li>children — количество детей в семье</li>
            <li>days_employed — общий трудовой стаж в днях</li>
            <li>dob_years — возраст клиента в годах</li>
            <li>education — уровень образования клиента</li>
            <li>education_id — идентификатор уровня образования</li>
            <li>family_status — семейное положение</li>
            <li>family_status_id — идентификатор семейного положения</li>
            <li>gender — пол клиента</li>
            <li>income_type — тип занятости</li>
            <li>debt — имел ли задолженность по возврату кредитов</li>
            <li>total_income — ежемесячный доход</li>
            <li>purpose — цель получения кредита</li>
        </ul>
        План выполнения проекта:
        <ol>
            <li>Определить и заполнить пропущенные значения;</li>
            <li>Проверить типы данных;</li>
            <li>Поработать с дубликатами;</li>
            <li>Выделить леммы;</li>
            <li>Категоризировтаь данные;</li>
            <li>Выявить зависимости возврата кредита вовремя от признаков;</li>
            <li>Сделать общий вывод.</li>
            <ol>
                </font>
            

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import math
import numpy as np
data = pd.read_csv('/datasets/data.csv')
data.info()
print(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее 

### Вывод

Файл содержит 12 колонок с информацией о заемщиках. Из них 2 столбца типа float64, 5 столбцов типа int64 и 5 столбцов типа object.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#print(data['dob_years'].value_counts())
#заполним нулевой возраст медианным значением
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].median()

#заменим отрицательные значения стажа положительными
data['days_employed'] = abs(data['days_employed']) 

#заменим отрицательные значения о количестве детей положительными
data['children'] = abs(data['children']) 

#print(data['days_employed'].isna().sum()) 
#заменим нулевые значения трудового стажа в днях на медианные значения с группировкой по количеству детей
for child in data['children'].unique():
    median = data.loc[data['children'] == child, 'days_employed'].median()
    data.loc[(data['days_employed'].isna()) & (data['children'] == child), 'days_employed'] = median

#сгруппируем значения по полу и образованию, чтобы заполнить пустые значения у total_income
medians = (data.groupby(['gender', 'education']).agg({'total_income':'median'}).rename(columns = {'total_income': 'median_total_income'}))

#добавляем новую колонку median_total_income к фрейму
data = data.merge(medians, on = ['gender', 'education'])

#там, где total_income - пустой, заполняем медианным значением
data.loc[data['total_income'].isna(), 'total_income'] = data.loc[data['total_income'].isna(), 'median_total_income']

#удалим колонку с медианой
data.drop(['median_total_income'], axis='columns', inplace=True)

#посмотрим колонку с количеством детей
#print(data['children'].value_counts())
#заменим число 20 на 2 - считаем, что при вводе ошиблись нулем
data.loc[data['children'] == 20, 'children'] = 2
#print(data['children'].value_counts())

print(data.head(30))

    children  days_employed  dob_years education  education_id  \
0          1    8437.673028       42.0    высшее             0   
1          0    2879.202052       43.0    высшее             0   
2          0    1844.956182       56.0    высшее             0   
3          1     717.274324       26.0    высшее             0   
4          0     650.587796       31.0    высшее             0   
5          1    6953.631655       50.0    высшее             0   
6          0    3480.072793       27.0    высшее             0   
7          0    2625.036979       52.0    высшее             0   
8          0     726.943771       43.0    высшее             0   
9          2    1680.557875       50.0    высшее             0   
10         0  373129.789654       63.0    высшее             0   
11         2    1680.557875       35.0    высшее             0   
12         1    1669.279681       40.0    высшее             0   
13         1    1669.174020       34.0    высшее             0   
14        

### Вывод

Вызвав к столбцу 'dob_years' метод value_counts(), который возвращает уникальные значения и количество их упоминаний, определим, сколько раз возраст был пропущен. Видим, что в 101 случае(около 0,4% от всех данных) он не был заполнен. Можно предположить, что не все пользователи любят заполнять колонку с возрастом.
Заполним его медианным значением, и в этом случае пожертвуем дисперсией. 

Методом isna() найдём все строки с пропусками в столбце 'days_employed'. В 2174 строках из 21525 общий трудовой стаж в днях пропущен. Заменим нулевые значения трудового стажа в днях на медианные значения с группировкой по количеству детей.

Вместе с этим столько же раз в столбце 'total_income' отсутствет информация о ежемесячном доходе. Рассчитаем и заполним их в зависимости от пола и образования. Отрицательные значения возьмем по модулю, так как общий трудовой стаж в днях не может быть отрицательным. Предположим, что исходные данные были импортированы с ошибками. Если взять отрицательные цифры по модулю, то они дают вполне реалистичную картину о трудовом стаже. То же самое проделаем с количеством детей - возьмем число по модулю.

Посмотрим колонку с количеством детей. Есть выделяющееся значение 20. Вероятно, ноль при вводе был добавлен ошибочно. Заменим 76 записей, в которых число детей равно 20, на 2.



### Замена типа данных

In [3]:
pd.options.display.float_format = '{:.2f}'.format
data['dob_years'] = data['dob_years'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data['children'] = data['children'].astype('int')
print(data.head(30))
data.info()

    children  days_employed  dob_years education  education_id  \
0          1           8437         42    высшее             0   
1          0           2879         43    высшее             0   
2          0           1844         56    высшее             0   
3          1            717         26    высшее             0   
4          0            650         31    высшее             0   
5          1           6953         50    высшее             0   
6          0           3480         27    высшее             0   
7          0           2625         52    высшее             0   
8          0            726         43    высшее             0   
9          2           1680         50    высшее             0   
10         0         373129         63    высшее             0   
11         2           1680         35    высшее             0   
12         1           1669         40    высшее             0   
13         1           1669         34    высшее             0   
14        

Трансформирмируем в целые столбцы стажа("days_employed"), возраста("dob_years"). В столбце с доходом("total_income") оставим 2 цифры после запятой.
Столбец "children" не может принимать непрерывные значения, поэтому конвертируем его в int. Остаьные типы данных считаем непротиворечивыми.

### Обработка дубликатов

In [4]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data.drop_duplicates().reset_index(drop=True)
#print(data.duplicated().sum())
#print(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,операции с жильем
2,0,1844,56,высшее,0,гражданский брак,1,F,компаньон,1,165127.91,покупка жилой недвижимости
3,1,717,26,высшее,0,женат / замужем,0,F,сотрудник,0,187863.24,строительство собственной недвижимости
4,0,650,31,высшее,0,гражданский брак,1,F,компаньон,0,754240.80,заняться образованием
...,...,...,...,...,...,...,...,...,...,...,...,...
21465,0,755,54,начальное,3,женат / замужем,0,M,сотрудник,0,172989.60,заняться образованием
21466,0,333641,56,начальное,3,вдовец / вдова,2,M,пенсионер,0,117294.50,ремонт жилью
21467,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.16,покупка недвижимости
21468,3,5968,36,ученая степень,4,женат / замужем,0,F,госслужащий,0,111392.23,покупка жилья


### Вывод

Для анализа значений столбцов был выбран метод value_counts(), так как его вывод более информативен. В столбце "education" много одинаковых по смыслу значений, но в разном регистре. Это возможно из-за того, что поле "Образование" при заполнении пользователяим не валидировалось и не приводило все заполненные данные в однообразный вид.  
Переведем все строковые значения в один регистр.
Дубликаты во всем датафрейме найдены с помощью метода duplicated() в сочетании с методом sum(). Он вернул 55 дубликатов. 
Для удаления использовался метод drop_duplicates(inplace = True).

### Лемматизация

In [5]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem() 
purposes = ' '.join(data['purpose'].unique())
lemmas = m.lemmatize(purposes)
print(lemmas)
print(Counter(lemmas))
data['lemmas_col'] = data['purpose'].apply(m.lemmatize)

def purpose_group(lemmas):
    if 'жилье' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'ремонт' in lemmas:
        return 'ремонт'
    return 'другое'
data['category'] = data['lemmas_col'].apply(purpose_group)
#print(data['category'].value_counts())
#удалим колонку с леммами
data.drop(['lemmas_col'], axis='columns', inplace=True)

['покупка', ' ', 'жилье', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'заниматься', ' ', 'образование', ' ', 'свадьба', ' ', 'автомобиль', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'свой', ' ', 'автомобиль', ' ', 'жилье', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'сыграть', ' ', 'свадьба', ' ', 'автомобиль', ' ', 'профильный', ' ', 'образование', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'сдача', ' ', 'сделка', ' ', 'с', ' ', 'подержанный', ' ', 'автомобиль', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'высокий', ' ', 'образование', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'покупка', ' ', 'свой', ' ', 'жилье', ' ', 'покупка'

### Вывод

Лемматизируем столбец "purpose" и найдем леммы, которые в нем встречаются чаще всего - это будут основные причины взятия кредитов. Создадим новый столбец "lemmas_col" с леммами причин взятия кредита.
Введем функцию purpose_group(), которая ищет леммы из списка основных причин в лемматизированном столбце 'lemmas_col'. В результате каждой строке соответствует одна причина взятия кредита.
После этого удалим столбец "lemmas_col", он больше не понадобится.

В итоге, мы выявили основные причины взятия кредитов и сопоставили каждое заявление какой-то из основных причин.

### Категоризация данных

In [6]:
#print(data.head(30))

#Узнаем, сколько видов образования всего - 5
print(len(data['education'].unique()))
education_dict = data[['education_id','education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict)

#Узнаем, сколько видов семейного положения всего - 5
print(len(data['family_status'].unique()))
family_status_dict = data[['family_status_id','family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict)

5
   education_id            education
0             0               высшее
1             1              среднее
2             2  неоконченное высшее
3             3            начальное
4             4       ученая степень
5
   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 4  не женат / не замужем
3                 2         вдовец / вдова
4                 3              в разводе


### Вывод

Составим словарь по паре education и education_id. Возьмем эти 2 столбца и выделим уникальные пары по параметру. 
Такую же операцию проделаем и с семейным статусом. 
В «словаре» видим большое количество дубликатов - удалим их. Применим drop_duplicates() и reset_index(). 
После удаления дубликатов представление данных с образованием и семейным статусом стало компактнее. Таблицу легко группировать по id.

Категоризация данных необходима для того, чтобы оптимизировать работу с основной таблицей и оптимизировать потребление памяти в данной таблице.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
print(data['children'].value_counts())
data.groupby('children')['debt'].agg(['count', 'mean'])
report1 = data.pivot_table(index = 'children', columns = 'debt', values = 'education', aggfunc = 'count').fillna(0)
report1.columns = ['no_debt','debt']
report1['percent'] = report1['debt'] / (report1['debt'] + report1['no_debt'])
display(report1.style.format({'percent':'{:.2%}', 'no_debt':'{:.0f}'}))


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


,no_debt,debt,percent
children,,,
0,13086,1063,7.51%
1,4420,445,9.15%
2,1929,202,9.48%
3,303,27,8.18%
4,37,4,9.76%
5,9,0,0.00%


### Вывод

Отслеживается зависимость между наличием детей и возвратом кредита в срок: наиболее рискованно выдавать кредиты семьям с 1-2 детьми. Данных по семьям с 4 и 5 детьми мало, чтобы сделать какие-то выводы.


In [8]:
report1_1 = data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count','sum','mean']).fillna(0)
display(report1_1)

,count,sum,mean
,debt,debt,debt
children,,,
0,14149,1063,0.08
1,4865,445,0.09
2,2131,202,0.09
3,330,27,0.08
4,41,4,0.10
5,9,0,0.00


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
report2 = data.pivot_table(index = 'family_status', columns = 'debt', values = 'education', aggfunc = 'count').fillna(0)
report2.columns = ['no_debt','debt']
report2['percent'] = report2['debt'] / (report2['debt'] + report2['no_debt'])
display(report2.style.format({'percent':'{:.2%}', 'no_debt':'{:.0f}'}))



,no_debt,debt,percent
family_status,,,
в разводе,1110,85,7.11%
вдовец / вдова,897,63,6.56%
гражданский брак,3789,388,9.29%
женат / замужем,11449,931,7.52%
не женат / не замужем,2539,274,9.74%


### Вывод

Отслеживается зависимость между семейным положением и возвратом кредита в срок - свободные люди и люди в гражданском браке более склонны к задолженностям по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
data['total_income_group'] = pd.cut(data['total_income'],[20000,100000,130000,160000,210000,np.inf])

report3 = data.pivot_table(index='total_income_group', columns='debt', values='total_income', aggfunc='count').fillna(0)
report3.columns = ['no_debt','debt']
report3['percent'] = report3['debt'] / (report3['debt'] + report3['no_debt'])
report3.style.format({'percent':'{:.2%}', 'no_debt': '{:.0f}'})



,no_debt,debt,percent
total_income_group,,,
"(20000.0, 100000.0]",4110,354,7.93%
"(100000.0, 130000.0]",4175,371,8.16%
"(130000.0, 160000.0]",3358,348,9.39%
"(160000.0, 210000.0]",3949,353,8.21%
"(210000.0, inf]",4192,315,6.99%


### Вывод

Сделаем категоризацию по доходу("total_income"). Для разделения на приблизительно равные категории используем метод .qcut(). Qcut режет набор данных по перцентилям, а они определяются приближенно. После этого заменим метод на .cut() и выделим промежутки.
Отслеживается зависимость между уровнем дохода и возвратом кредита в срок - люди со средним доходом (130-160 т.р.) - самые рискованные плательщики. Наименее рискованные плательщики - люди с высоким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
report4 = data.pivot_table(index = 'category', columns = 'debt', values = 'education', aggfunc = 'count').fillna(0)
report4.columns = ['no_debt','debt']
report4['percent'] = report4['debt'] / (report4['debt'] + report4['no_debt'])
display(report4.style.format({'percent':'{:.2%}', 'no_debt':'{:.0f}'}))

report5 = data.pivot_table(index = 'income_type', columns = 'debt', values = 'total_income', aggfunc = 'count').fillna(0)
report5.columns = ['no_debt','debt']
report5['percent'] = report5['debt'] / (report5['debt'] + report5['no_debt'])
display(report5.style.format({'percent':'{:.2%}', 'no_debt':'{:.0f}'}))

,no_debt,debt,percent
category,,,
автомобиль,3912,403,9.34%
недвижимость,10058,782,7.21%
образование,3652,370,9.20%
свадьба,2162,186,7.92%


,no_debt,debt,percent
income_type,,,
безработный,1,1,50.00%
в декрете,0,1,100.00%
госслужащий,1373,86,5.89%
компаньон,4709,376,7.39%
пенсионер,3640,216,5.60%
предприниматель,2,0,0.00%
сотрудник,10058,1061,9.54%
студент,1,0,0.00%


### Вывод

Наиболее рискованные цели кредита - это категории, связанные с автомобилями и образованием. Они наиболее подвержены невозврату в срок.

Дополнительно посмотрим зависимость от типа занятости. Данные о типах занятости "безработный", "в декрете", "предприниматель", "студент" - неинформативны, поскольку их слишком мало. Из оставшихся типов занятости наиболее рискованный тип по выплате кредита в срок - это сотрудники компаний.

### Шаг 4. Общий вывод

Основная задача этого проекта - выделить ненадежные группы заемщиков. <font color="#120a8f">Входные данные от банка — статистика о платёжеспособности клиентов (21525 строк с данными о клиентах). Исследованы разные измерения для того, чтобы выяснить, кто отдает кредиты реже - семейное положение, цели взятия кредита, количество детей, ежемесячный доход и тип занятости.

В результате выявлена зависимость между наличием детей и возвратом кредита в срок: наиболее рискованно выдавать кредиты семьям с 1-2 детьми (9-10% процентов таких клиентов задерживают выплаты по кредитам). Категории с 4 и 5 детьми имеют недостаточные выборки для анализа и вывода.

Также отслеживается зависимость между семейным положением и возвратом кредита в срок - свободные люди и люди в гражданском браке более склонны к задолженностям по кредиту (~10%).

Выявлена взаимосвязь между уровнем дохода и возвратом кредита в срок - люди со средним доходом (130-160 т.р.) - самые рискованные плательщики (их ~10%). Наименее рискованные плательщики - люди с высоким доходом (93% возвращает кредиты в срок).

В результате исследования зависимости цели кредита и его возврата в срок, можно сделать вывод, что наиболее рискованные цели кредита - это категории, связанные с автомобилями и образованием. Они наиболее подвержены невозврату в срок (9-10% клиентов не возвращает кредит в срок).

Дополнительно изучена зависимость от типа занятости. Данные о типах занятости "безработный", "в декрете", "предприниматель", "студент" - неинформативны, поскольку их слишком мало. Из оставшихся типов занятости наиболее рискованный тип по выплате кредита в срок - это сотрудники компаний (~10% имеют трудности с возвратом кредита в срок). Наименее рискованные - это пенсионеры или госслужащие (94% возвращает кредит вовремя).</font>

По полученным данным можно сделать вывод, что при любых условиях около 90% клиентов возвращают кредит в срок. Поэтому кредитование является довольно популярным бизнесом. 
<font color="#120a8f">Резюмируя, предоставим краткие рекомендации и портреты "хороших" клиентов и "недотянувших до хороших".

Хорошие клиенты, которые возвращают кредит в срок:
- клиенты, не имеющие детей. 92,5% процентов таких клиентов возвращают кредит в срок;
- 93% клиентов, находящихся в семейном положении "вдовец/вдова" и "женат/замужем" своевременно возвращают кредит;
- наименее рискованные плательщики - люди с высоким уровнем дохода (от 210 т.р.) - 93% таких клиентов возвращают кредиты в срок;
- наименее рискованные цели взятия кредита - это недвижимость и свадьба. 92% клиентов не задерживают с выплатами по кредитам, взятых для таких целей;
- клиенты, являющиеся пенсионерами или госслужащими, наиболее успешно возвращают кредит в срок (таких 94%).
</font>

Из тех, кто отдает кредиты в срок наименее успешно, это:
- клиенты с 1-2 детьми. <font color="#120a8f">9-10% процентов таких клиентов задерживают выплаты по кредитам;</font>
- неженатые/незамужние клиенты, а также клиенты, состоящие в гражданском браке. <font color="#120a8f">Около 10% таковых склонны к задолженностям по кредитам;</font>
- клиенты с доходом 130-160 т.р. <font color="#120a8f">Около 10% таких клиентов имеют задолженности по кредитам. Относительно рядом по показателям с ними и клиенты, имеющие доход чуть ниже (100-130 т.р.) и чуть выше (160-210 т.р.) - около 8% таких имеют затруднения с возвратом кредита в срок;</font>
- клиенты, берущие кредит на автомобили и образование.<font color="#120a8f"> Это наиболее рискованные цели кредита. 9-10% людей, берущих кредиты для этих целей, не возвращают его в срок;</font>
- клиенты с типом занятости "сотрудник".<font color="#120a8f"> Около 10% лиц, являющихся сотрудниками компаний, не справляются с возвратом кредита в срок.</font>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.